In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext tensorboard

In [ ]:
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
random_seed = 9265

from model import SkipGramModel, NCELoss
from data import Train_Dataset
from skipgram import EmbTrainer
from utils import get_loaders, visualize_embeddings

In [ ]:
# dataset = Train_Dataset('datasets/open_subtitles_pl_georgia/datasetSentences.csv', window_size=5)
loaders = get_loaders(dataset, batch_size=16)

epochs = 300
model = SkipGramModel(v_size=len(dataset), emb_dim=100)
optim = torch.optim.SGD(model.parameters(), lr=0.001)
params_trainer = {
    'model': model,
    'loaders': loaders,
    'criterion': NCELoss(),
    'optim': optim,
    'scheduler': torch.optim.lr_scheduler.CosineAnnealingLR(optim, epochs),
    'verbose': True,
    'device': device
#     'params_clearml': params_clearml,
#     'is_tensorboard': True
}

trainer = EmbTrainer(**params_trainer)

In [ ]:
trainer.loop(0, epochs)

In [ ]:
word2Ind = {word: i for i, word in dataset.Ind2word.items()}

chosen_words = [
    "świetny", "głupi", "wspaniały", "spoko", "ekstra",
    "słodki", "nudny",   "zły", 
    "artyleria", "generał", 'porucznik', "sierżant", "wojsko",
#     "dobrze"
] 
chosen_idxs = [word2Ind[word] for word in chosen_words]

visualize_embeddings(model.emb.weight.detach().cpu().numpy(), chosen_idxs, dataset.Ind2word)

In [ ]:
class NCELossClassifier(torch.nn.Module):
    def __init__(self, num_class=10):
        super().__init__()
        self.num_class = num_class

    def forward(self, y_pred, y_true):
        '''
        y_pred : torch.tensor (b, c)
        y_true : torch.tensor (b, c)
        '''
        y_true = F.one_hot(y_true, num_classes=self.num_class)
        pos = torch.log(torch.sigmoid(y_pred * y_true))
        negs = torch.log(torch.sigmoid(- y_pred * (1 - y_true)))
        loss = - pos.mean() - negs.mean()
        return loss